# <center>**WORD VECTORS**</center>

In [ ]:
import os
import gensim
from gensim.models import Word2Vec
import glob
import nltk

from lxml import etree as ET
import lxml.html
import string
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
!gdown 1KU8Jx7On0xmDoIGegxHAnliS8wk2rUq3

In [ ]:
!unzip "/content/auteurs.zip"

In [ ]:
dumas="auteurs/flaubert/"
balzac="auteurs/balzac/"

In [ ]:
def strip_ns_prefix(tree):
    query = "descendant-or-self::*[namespace-uri()!='']"
    for element in tree.xpath(query):
        element.tag = ET.QName(element).localname
    return tree

In [ ]:
if balzac != "":
    files = glob.iglob(balzac + '/**/*.xml', recursive=True)
    sentences = []

    for filename in files:
        print(filename)
        parser = ET.XMLParser(remove_blank_text=True, resolve_entities=False, encoding='utf8')
        tree = strip_ns_prefix(ET.parse(filename, parser))

        words = tree.xpath(".//wf/@lemma")

        sentence = []
        for word in words:
            if word != ".":
                sentence.append(word)
            else:
                sentences.append(sentence + [word])
                sentence = []

In [ ]:
print(len(sentences))
print(sentences[5])

## Construction du modèle vectoriel

In [ ]:
model = Word2Vec(sentences, min_count=2, max_vocab_size=10000, negative=10, epochs=100)

In [ ]:
model.wv.save("/content/model_balzac.bin")

In [ ]:
print(model.wv.index_to_key)

In [ ]:
#Paris is to France what London is to what ? model.wv.most_similar(positive=['Londres', 'France'], negative=['Paris'],topn=5)
#King is to man what Queen is to what ? model.wv.most_similar(positive=['reine', 'homme'], negative=['roi'],topn=5)
model.wv.most_similar(positive=['reine', 'homme'], negative=['roi'],topn=10)

In [ ]:
model.wv.most_similar('esprit',topn=10)

## Visualization

In [ ]:
!gdown 15XszQYD6Ld_eKLPidzETcTQX9-H053xV

Downloading...
From: https://drive.google.com/uc?id=15XszQYD6Ld_eKLPidzETcTQX9-H053xV
To: /content/stopwords.txt
100% 4.59k/4.59k [00:00<00:00, 16.4MB/s]


In [ ]:
stops = open("/content/stopwords.txt", encoding="utf-8").read().split("\n")

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

In [ ]:
nb_mots_a_visualiser = 500
words = [word for word in model.wv.index_to_key if word not in stops and word not in string.punctuation][:500]
word_vectors = np.array([model.wv[word] for word in words])

tsne = TSNE(n_components=3, perplexity=30, learning_rate=200,random_state=0)
vectors_3d = tsne.fit_transform(word_vectors)

import pandas as pd
df = pd.DataFrame(vectors_3d, columns=['x', 'y', 'z'])
df['word'] = words

fig = px.scatter_3d(df, x='x', y='y', z='z', text='word', title="Visualisation 3D des Vecteurs Word2Vec")
fig.show()

In [ ]:
fig.write_html("/content/word2vec_visualization.html")